In [4]:
import json
import requests

# Querying the Drugmonizome APIs
Drugmonizome includes two API that users can interact with: (1) the metadata API (https://maayanlab.cloud/drugmonizome/metadata-api), and (2) the data API (https://maayanlab.cloud/drugmonizome/data-api). The metadata API provides fast full-text search and field comparison filtering of the metadata as well as metadata aggregations for statistical summaries. Users can query the following models (1) resources, (2) libraries, (3) signatures, (4) entities. Meanwhile, the data API performs real-time drug set enrichment analysis. Here we show some of the basic use cases for the two APIs.

## Metadata Search
Users can query for metadata using JSON formatted loopback queries (https://loopback.io/doc/en/lb3/Querying-data.html).

**Method** `GET`

URL `/<model>`

**PARAMETERS**

**filter** stringified JSON query

**RETURNS**

JSON list of all matches

### Full Text Search

In [5]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/signatures"
term = "headache"

query = {
    "where": {
        "meta": {"fullTextSearch": term}     
    },
    "limit": 1
}
res = requests.get(METADATA_API+ endpoint , params={"filter": json.dumps(query)})
print(json.dumps(res.json(),indent=4))

[
    {
        "$validator": "/dcic/signature-commons-schema/v5/core/signature.json",
        "id": "dc73bc80-3cf2-4c9c-8692-7de0ea39412d",
        "library": "1ab551d0-f689-4358-a51b-3dc74b4cd6e1",
        "meta": {
            "Term": [
                {
                    "Name": "tension headache",
                    "Accession": "C0033893"
                }
            ],
            "Filename": "SIDER_side_effects_drugsetlibrary",
            "Organism": "Homo sapiens",
            "$validator": "https://raw.githubusercontent.com/MaayanLab/Drugmonizome/master/drugmonizome-schemas/meta/signature.json",
            "Number_of_drugs": 17,
            "Association_Type": "side effects"
        }
    }
]


Alternatively, users can also perform a POST request using the `/find` endpoint

**Method** `POST`

URL `/<model>/find`

**PARAMETERS**

**filter** stringified JSON query

**RETURNS**

JSON list of all matches

### Filtering by fields

In [6]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/libraries/find"
term = "side effects"

query = {
    "filter":{
        "where": {
            "meta.Association_Type": term     
        },
        "limit": 1
    }
}
res = requests.post(METADATA_API+ endpoint, json=query)
print(json.dumps(res.json(),indent=4))

[
    {
        "$validator": "/dcic/signature-commons-schema/v5/core/library.json",
        "id": "1ab551d0-f689-4358-a51b-3dc74b4cd6e1",
        "resource": "0dd36110-b67b-4546-ac36-974a6432896a",
        "dataset": "SIDER_side_effects_drugsetlibrary",
        "dataset_type": "geneset_library",
        "meta": {
            "DOI": "10.1038/clpt.2012.96",
            "Icon": "https://raw.githubusercontent.com/MaayanLab/Drugmonizome/master/drugmonizome-schemas/images/sider.png",
            "PMID": "26481350",
            "Year": "2015",
            "Version": "2019",
            "Filename": "SIDER_side_effects_drugsetlibrary",
            "Organism": "Homo sapiens",
            "$validator": "https://raw.githubusercontent.com/MaayanLab/Drugmonizome/master/drugmonizome-schemas/meta/library.json",
            "Description": "Approved drug side effects mined from FDA package inserts and public documents",
            "Library_name": "SIDER Side Effects",
            "Download_link": "htt

## Aggregating Metadata

### Counting Results
**Method** `GET`

URL `/<model>/count`

**PARAMETERS**

**filter** stringified JSON query

**RETURNS**

The number of matching results

In [7]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/signatures/count"
term = "STAT3"

query = {
    "meta": {"fullTextSearch": term}     
}
res = requests.get(METADATA_API+ endpoint,  params={"where": json.dumps(query)})
print(json.dumps(res.json(),indent=4))

{
    "count": 12
}


### Counting Top Terms
**Method** `GET`

URL `/<model>/value_count`

**PARAMETERS**

**filter** stringified JSON query

**RETURNS**

JSON list of value counts

In [12]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/signatures/value_count"
term = "headache"

query = {
    "fields": ["meta.Filename"],
    "where": {
        "meta": {"fullTextSearch": term}     
    }
}
res = requests.get(METADATA_API+ endpoint,  params={"filter": json.dumps(query)})
print(json.dumps(res.json(),indent=4))

{
    "meta.Filename": {
        "PharmGKB_OFFSIDES_side_effects_drugsetlibrary": 7,
        "SIDER_side_effects_drugsetlibrary": 5,
        "SIDER_indications_drugsetlibrary": 3,
        "L1000FWD_predicted_side_effects": 2
    }
}


## Signature Search

Performing the signature search requires a few steps that requires the users to query both the metadata and data API.

### Getting the list of datasets

Users must specify the dataset they want to perform drug set enrichment analysis on. To figure out the list of available datasets, one can use the /listdata endpoint of the data api.


**Method** `GET`

URL `/listdata`

**RETURNS**

A JSON list of datasets stating the identifier for the dataset and the datatype

In [14]:
DATA_API = "https://maayanlab.cloud/drugmonizome/data-api/api/v1"
endpoint = "/listdata"

res = requests.get(DATA_API+ endpoint)
print(json.dumps(res.json(),indent=4))

{
    "repositories": [
        {
            "datatype": "geneset_library",
            "uuid": "KinomeScan_kinase_drugsetlibrary"
        },
        {
            "datatype": "geneset_library",
            "uuid": "L1000FWD_GO_Cellular_Component_drugsetlibrary_up"
        },
        {
            "datatype": "geneset_library",
            "uuid": "L1000FWD_KEGG_Pathways_drugsetlibrary_down"
        },
        {
            "datatype": "geneset_library",
            "uuid": "L1000FWD_signature_drugsetlibrary_up"
        },
        {
            "datatype": "geneset_library",
            "uuid": "Geneshot_associated_drugsetlibrary"
        },
        {
            "datatype": "geneset_library",
            "uuid": "Geneshot_predicted_generif_drugsetlibrary"
        },
        {
            "datatype": "geneset_library",
            "uuid": "SIDER_indications_drugsetlibrary"
        },
        {
            "datatype": "geneset_library",
            "uuid": "L1000FWD_GO_Molecular_Functi

**The datatype in the `/listdata` endpoint is "geneset_library". This is part of the core schema of the Signature Commons data-api and is currently immutable. These are, however, drug set libraries**

### Converting drugs and small molecules to UUIDs
Compounds should be converted to UUIDs before sending them to the data api for signature search. To do this, we need to perform a metadata search using the `/entities` endpoint

Drugs can be searched by **name**, **InChI Key**, **DrugBankIDs** or **BRD-IDs**, and **SMILES strings**

#### Querying by name

In [17]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/entities/find"
drugs = ['dexamethasone','daunorubicin','doxorubicin']

query = {
    "filter":{
        "where": {
            "meta.Name": {
                "inq": drugs
            }     
        }
    }
}

res = requests.post(METADATA_API+ endpoint, json=query)
drugs_ids = [i["id"] for i in res.json()]
print(drugs_ids)

['38c486b7-8ccb-47a2-b592-69bc244e214b', '39fe27fe-795e-4d03-b556-7bf1cc6664f3', '31afacf7-5514-4602-856d-3d1d0f899ef1', '8edc5e45-9bc6-4782-97da-b34d9da5e927', '14fef7a1-7c7b-413c-b72e-5579f01521c8', '57063362-21e5-46f9-a34e-1117e2ae1e3f', 'af7d9ae5-7970-42be-b2a6-e45acfe873b2', '3a5711bd-1837-46ea-bc33-d564181beade']


When querying entities by name, it may be the case that entities share the same name despite having unique InChI Keys that denote variable stereochemistry. Querying by name or SMILES string will be less specific and retrieve all records of an entity within the database. If the user is concerned with stereochemistry or other specific attributes of a molecule, they may query by InChI key or specific DrugBankIDs or BRD-IDs.

#### Querying by InChI Key

In [20]:
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/entities/find"
drugs = ['UREBDLICKHMUKA-CXSFZGCWSA-N','STQGQHZAVUOBTE-VGBVRHCVSA-N','AOJJSUZBOXZQNB-TZSSRYMLSA-N']

query = {
    "filter":{
        "where": {
            "meta.InChI_key": {
                "inq": drugs
            }     
        }
    }
}

res = requests.post(METADATA_API+ endpoint, json=query)
drugs_ids = [i["id"] for i in res.json()]
print(drugs_ids)

['55b26a20-e390-49e2-9063-2a26f31b50df', '38c486b7-8ccb-47a2-b592-69bc244e214b', '39fe27fe-795e-4d03-b556-7bf1cc6664f3']


Compared to the previous case where we queried by name, when we query by InChI key we are presented solely with the UUIDs that correspond to that InChI Key

### Enriching a Drug Set

**Method** `POST`

**URL** `/enrich/overlap`

**Query**

```
{
  "database": "string",
  "entities": [
    "string"
  ],
  "offset": 0,
  "limit": 0
}
```

**RETURNS**

The analysis results

In [22]:
## Convert drug names to uuids
METADATA_API = "https://maayanlab.cloud/drugmonizome/metadata-api"
endpoint = "/entities/find"
drugs = ['mefloquine','hydroxychloroquine','chloroquine','remdesivir','proscillaridin','niclosamide','hexachlorophene','lopinavir','amodiaquine','bazedoxifene','anidulafungin','cyclosporine','loperamide','ivacaftor','abemaciclib','almitrine','camostat']

query = {
    "filter":{
        "where": {
            "meta.Name": {
                "inq": drugs
            }     
        }
    }
}

res = requests.post(METADATA_API+ endpoint,  json=query)
drugs_ids = [i["id"] for i in res.json()]

# Signature Search
DATA_API = "https://maayanlab.cloud/drugmonizome/data-api/api/v1"
endpoint = "/enrich/overlap"

query = {
    "database": "L1000FWD_GO_Biological_Processes_drugsetlibrary_up",
    "entities": drugs_ids,
    "limit": 5
}

res = requests.post(DATA_API + endpoint, json=query)
signatures = [i["uuid"] for i in res.json()["results"]]

# Get metadata for signatures
endpoint = "/signatures/find"
query = {
    "filter":{
        "where": {
            "id": {
                "inq": signatures
            }     
        }
    }
}

res = requests.post(METADATA_API+ endpoint,  json=query)
print(json.dumps(res.json(),indent=4))

[
    {
        "$validator": "/dcic/signature-commons-schema/v5/core/signature.json",
        "id": "6ab56d97-626d-4072-896f-c2cca56ab303",
        "library": "90e4bbcf-13ce-4154-9735-ac0a84cae23f",
        "meta": {
            "Term": [
                {
                    "Name": "negative regulation of cell proliferation (GO:0008285)",
                    "Accession": "GO:0008285"
                }
            ],
            "Filename": "L1000FWD_GO_Biological_Processes_drugsetlibrary_up",
            "Organism": "Homo sapiens",
            "Direction": "up",
            "$validator": "https://raw.githubusercontent.com/MaayanLab/Drugmonizome/master/drugmonizome-schemas/meta/signature.json",
            "Number_of_drugs": 553,
            "Association_Type": "Gene Ontology terms"
        }
    },
    {
        "$validator": "/dcic/signature-commons-schema/v5/core/signature.json",
        "id": "e3182072-1ce1-4bd1-a50f-df71c31bd8fc",
        "library": "90e4bbcf-13ce-4154-9735-ac0a